<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>


<br>

# <font color="#76b900">**Notebook 2:** LLM 服務與 AI 基礎模型</font>

<br>

在這個 notebook 中，我們將探索 LLM 服務！我們將討論在邊緣裝置(Edge Device)上部署架構(Deployment) LLM 的優缺點，以及透過可擴展的伺服器部署架構(Deployment)（如透過 NVIDIA AI Foundation 端點(Endpoints)存取的服務）向終端使用者提供強大模型的方法。

<br>

### **學習目標：**

-   了解在本地與可擴展雲端環境中運行 LLM 服務的優缺點。
-   熟悉 AI 基礎模型端點(Endpoints)方案(Schemes)，包括：
    -   由 `curl` 和 `requests` 等套件促進的原始低階連接介面
    -   為使此介面與 LangChain 等開源軟體無縫運作而創建的想法(abstractions)
-   熟練地從端點(Endpoints)池中檢索(Retrieval) LLM 生成結果，並能夠選擇模型子集來建構您的軟體。
<br>

### **值得思考的問題：**

1.  您應該為開發 LLM 層層結構(Stack)的人提供什麼樣的模型存取權限，這與您需要為 AI 驅動的網路應用程式終端使用者提供的存取權限相比如何？

2.  在考慮支援哪些裝置時，您對其本地計算資源做出了什麼樣的嚴格假設，以及您應該實施什麼類型的備用方案？

    -   如果您想為客戶提供具有私有 LLM 部署架構(Deployment)存取權限的 jupyter labs 介面會如何？

    -   如果現在您想用您的私有 LLM 部署架構(Deployment)支援他們的本地 jupyter lab 環境會如何？

    -   如果您決定支援嵌入式裝置（即 Jetson Nano），是否需要改變任何東西？

3.  **[較困難]** 假設您在雲端環境中的自己計算實體(Instance)上部署架構(Deployment)了 Stable Diffusion、Mixtral 和 Llama-13B，共享相同的 GPU 資源。您目前沒有 Stable Diffusion 的商業使用案例，但您的團隊正在用其他兩個進行 LLM 應用程式實驗。您應該從部署架構(Deployment)中移除 Stable Diffusion 嗎？

<br>







<br>

## **第一部分**：將大型模型導入您的環境

回想上一個 notebook，我們目前的環境在分配的雲端實體(Instance)上運行著幾個微服務(MICROSERVICES)：`docker-router`、`jupyter-notebook-server`、`frontend` 和 `llm-service`（以及其他服務）。

-   **jupyter-notebook-server**：運行此 jupyter labs 會話並託管(Hosting)我們 python 環境的服務。
-   **docker_router**：幫助我們至少觀察和監控微服務(MICROSERVICES)的服務。
-   **frontend**：為我們提供簡單聊天介面的運行中的(Live)網站微服務(MICROSERVICES)。

這個 notebook 將更多關注 `llm-service` 微服務(MICROSERVICES)，您將使用它（至少在幕後）與一系列[**基礎模型**](https://www.nvidia.com/en-us/ai-data-science/foundation-models/)介面！具體來說，您將使用[**NVIDIA AI 基礎模型**](https://catalog.ngc.nvidia.com/)的子集來原型化 AI 啟用的管線(Pipeline)並流程協調管理(Orchestration)重要的自然語言支援應用程式。



$$---$$



在幾乎每個領域中，部署架構(Deployment)大規模深度學習模型都是一項常見但具有挑戰性的任務。今天的模型，如 Llama 2（700 億參數）或像 Mixtral 7x8B 這樣的集成模型，是先進訓練方法、龐大資料資源和強大計算系統的產物。幸運的是，這些模型已經被訓練過，許多使用案例已經可以透過現成的解決方案實現。然而，真正的障礙在於有效地託管(Hosting)這些模型。

**大型模型的部署架構(Deployment)情境：**

1.  **高端資料中心部署架構(Deployment)：**
> 在配備 NVIDIA [A100](https://www.nvidia.com/en-us/data-center/a100/)/[H100](https://www.nvidia.com/en-us/data-center/h100/)/[H200](https://www.nvidia.com/en-us/data-center/h200/) 等 GPU 的資料中心層層結構(Stack)上的未壓縮、未量化模型，以促進快速推論(Inference)和實驗。
>
> -   **優點**：非常適合可擴展的部署架構(Deployment)和實驗，這個層層結構(Stack)非常適合大型訓練工作流程或同時支援多個使用者或模型。
> -   **缺點：** 除非使用案例涉及模型訓練/微調或與較低階模型組件介面，否則為您服務的每個使用者分配此資源是低效的。

2.  **適度資料中心/專業消費者硬體部署架構(Deployment)：**

> 量化和進一步最佳化的模型可以在更保守的資料中心 GPU（如 [L40](https://www.nvidia.com/en-us/data-center/l40/)/[A30](https://www.nvidia.com/en-us/data-center/products/a30-gpu/)/[A10](https://www.nvidia.com/en-us/data-center/products/a10-gpu/)）上運行（每個實體(Instance)一到兩個），甚至在一些現代消費者 GPU（如高 VRAM [RTX 40 系列 GPU](https://www.nvidia.com/en-us/geforce/graphics-cards/40-series/)）上運行。
>
> -   **優點：** 這種設置平衡了推論(Inference)速度與單使用者應用程式的可管理限制。這些會話也可以按使用者部署架構(Deployment)，一次運行一到兩個大型模型，並可原始存取模型內部（即使它們需要量化）。
> -   **缺點：** 為每個使用者部署架構(Deployment)實體(Instance)在規模上仍然昂貴，儘管對於某些利基工作負載可能是合理的。或者，假設使用者可以在其本地環境中存取這些資源可能是不合理的。


3.  **消費者硬體部署架構(Deployment)：**

> 儘管在透過神經網路傳播資料的能力方面嚴重受限，但大多數消費者硬體確實具有圖形使用者介面 (GUI)、具有網際網路存取的網路瀏覽器、一定量的記憶體（可以安全地假設至少 1 GB）和相當強大的 CPU。
>
> -   **缺點：** 目前大多數硬體在任何配置下一次無法運行超過一個本地大型模型，即使運行一個模型也需要大量的資源管理和最佳化限制。
> -   **優點：** 在考慮您的服務應該支援什麼樣的使用者時，這是一個合理且包容的起始假設。
>













在這門課程中，您的環境將非常代表典型的消費者硬體；儘管我們可以用微服務(MICROSERVICES)啟動和原型化，但我們受到僅 CPU 計算環境的約束，這將難以運行 LLM 模型。雖然這是一個重大限制，但我們仍然能夠透過以下方式利用快速 LLM 功能：

-   存取具有計算能力的服務來託管(Hosting)大型模型。

-   用於命令輸入(Intake)和結果檢索(Retrieval)的簡化介面。

有了我們在微服務(MICROSERVICES)和基於連接埠的連接方面的基礎，我們已經準備好探索為我們的開發環境獲得 LLM 存取的有效介面選項(interfacing options)！



<br>

## **第二部分：** 託管(Hosting)大型模型服務


在我們追求在像我們這樣的資源受限環境（通常僅使用 CPU ）中提供大型語言模型 (LLM) 存取的過程中，我們將評估各種託管(Hosting)選項：

**黑盒託管(Hosting)模型：**

> 像 [**OpenAI**](https://openai.com/) 這樣的服務提供 API 來與 GPT-4 等黑盒模型互動。這些強大、整合良好的服務可以為複雜的管線(Pipeline)提供簡單的介面，自動追蹤記憶體、呼叫額外的模型，並根據需要整合多模態介面以簡化典型的使用情境。同時，它們保持操作不透明性，通常缺乏自託管(Hosting)的直接路徑。
>
> -   **優點：** 開箱即用易於使用，對普通使用者的進入門檻較低。
> -   **缺點：** 黑盒部署架構(Deployment)存在潛在的隱私問題、有限的客製化以及規模成本影響。



**自託管(Hosting)模型：**

> 在幾乎所有規模化模型部署架構(Deployment)的幕後，都有一個或多個在資料中心運行的巨型模型，擁有可擴展的資源和閃電般快速的頻寬。儘管對於大規模部署架構(Deployment)大型模型並保持對提供介面的強力控制是必要的，但這些系統通常需要專業知識來設置，並且通常不適合僅為一個人支援非開發者工作流程。這樣的系統更適合同時支援許多使用者、多個模型和自訂介面。
>
> -   **優點：** 它們提供整合自訂資料集和 API 的能力，主要設計用於同時支援眾多使用者。
> -   **缺點：** 這些設置需要技術專業知識來設置和正確配置。


為了獲得兩全其美的效果，我們將利用 [**NVIDIA NGC 服務**](https://www.nvidia.com/en-us/gpu-cloud/)。NGC 提供一套用於設計和部署架構(Deployment) AI 解決方案的開發者工具。對我們需求的核心是 [NVIDIA AI 基礎模型](https://www.nvidia.com/en-us/ai-data-science/foundation-models/)，這些是預調整和預最佳化的模型，設計用於輕鬆開箱即用的可擴展部署架構(Deployment)（按原樣或進一步客製化）。此外，NGC 託管(Hosting)可存取的模型端點(Endpoints)，用於在[可擴展的 DGX 加速計算環境](https://www.nvidia.com/en-us/data-center/dgx-platform/)中查詢運行中的(Live)基礎模型。




<br>

## **第三部分：** 開始使用託管(Hosting)推論(Inference)

**當部署架構(Deployment)模型進行規模化推論(Inference)時，您通常需要採取的步驟如下：**
-   識別您希望使用者存取的模型，並分配資源來託管(Hosting)它們。
-   找出您希望使用者擁有什麼樣的控制，並公開他們存取它的方式。
-   創建監控方案(Schemes)來追蹤/限制使用，並設置系統來根據需要擴展和節流。

對於這門課程，您將使用由 NVIDIA 部署架構(Deployment)的模型，這些模型作為 [**NVIDIA NIM 微服務(MICROSERVICES)**](https://www.nvidia.com/en-us/ai/) 託管(Hosting)。NIM 生態系統由最佳化運行 AI 工作負載以進行規模化推論(Inference)部署架構(Deployment)的微服務(MICROSERVICES)組成。它們對本地推論(Inference)運作良好並提供標準化 API，但主要設計用於在規模化環境中特別良好地運作。這些特定模型部署架構(Deployment)在 NVIDIA DGX Cloud 上作為共享函式(function)，並透過 OpenAPI 風格的 API 閘道器推廣。讓我們解釋這意味著什麼：

**在叢集式(Clustering)端：** 這些微服務(MICROSERVICES)託管(Hosting)在 Kubernetes 支援的平台上，該平台將負載擴展到最小和最大數量的 DGX 節點(Node)，並在單一函式(function)後面提供。換句話說：

-   大型語言模型被下載到並部署架構(Deployment)在 **GPU 啟用的計算節點(Node)**（即強大的 CPU 和 4xH100-GPU 環境，在 DGX Pod 中物理整合）。
-   啟動時，會啟動這些計算節點(Node)的選擇，這樣每當使用者向函式(function)發送請求時，其中一個節點(Node)將接收請求。
    -   Kubernetes 將適當地路由這個流量。如果有空閒的計算節點(Node)，它將接收流量。如果它們都在工作，請求將被排隊，節點(Node)將盡快接收它。
    -   在我們的情況下，這些節點(Node)仍然會很快接收請求，因為啟用了即時動態(in-flight)批次處理，意味著每個節點(Node)可以在完全「滿載」之前一次接收多達 256 個活躍請求。（256 是部署架構(Deployment)上的超參數）。
-   隨著負載開始增加，自動擴展將啟動，更多節點(Node)將被啟動以避免請求處理延遲。

以下圖像顯示了具有自訂（非 OpenAPI）API 的任意函式(function)調用。這是公共端點(Endpoints)最初推廣的方式，但現在變成是常見的實作方式。

<!-- > <img style="max-width: 1000px;" src="imgs/ai-playground-api.png" /> -->
<!-- > <img src="https://drive.google.com/uc?export=view&id=1ckAIZoy7tvtK1uNqzA9eV5RlKMbVqs1-" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/ai-playground-api.png" width=800px/>




**在閘道器端：** 為了使這個 API 更標準，使用 API 閘道器伺服器將這些函式(function)聚合在稱為 OpenAPI 的通用 API 後面。這個規範被包括 OpenAI 在內的許多人訂閱(subscribed)，所以使用 OpenAI 客戶端是一個有效的介面：

<!-- > <img style="max-width: 800px;" src="imgs/mixtral_api.png" /> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/mixtral_api.png" width=800px/>








對於這門課程，您將希望使用連接到稱為 LangChain 的 LLM 流程協調管理(Orchestration)框架的更專業介面（稍後詳述）。在您這端，您將使用更客製化的介面，如來自 [`langchain_nvidia_ai_endpoints`](https://python.langchain.com/docs/integrations/chat/nvidia_ai_endpoints/) 函式庫(library)的 `ChatNVIDIA`。*稍後詳述。*



**在使用者端：** 將這些端點(Endpoints)整合到您的客戶端中，您可以設計整合、管線(Pipeline)和使用者體驗，利用這些生成式 AI 功能為您的應用程式賦予推理和生成能力。這樣應用程式的一個常見範例是 [**OpenAI 的 ChatGPT**](https://chat.openai.com/)，它是包括 GPT4、Dalle 和其他端點(Endpoints)的流程協調管理(Orchestration)。儘管它有時看起來像一個單一的智慧模型，但它僅僅是模型端點(Endpoints)的聚合，加上軟體工程來幫助管理狀態和脈絡資訊(Context)控制。這將在整個課程中得到加強，到最後您應該對如何為任意使用案例製作類似的聊天助手有概念。

<!-- > <img style="max-width: 700px;" src="imgs/openai_chat.png" /> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/openai_chat.png" width=700px/>




<br>

## **第四部分：\[練習\]** 試用基礎模型端點(Endpoints)



在這個部分，您將最終能夠與您的 LLM 端點(Endpoints)互動！

**從您自己的環境：** 您會想要前往 [`build.nvidia.com`](https://build.nvidia.com/) 並找到您想要使用的模型。例如，您可以前往 [**MistralAI 的 Mixtral-8x7b 模型**](https://build.nvidia.com/mistralai/mixtral-8x7b-instruct) 來查看如何使用模型的範例、進一步閱讀的連結，以及一些按鈕，如「申請自託管(Hosting)」和「獲取 API 金鑰」。

-   點擊 **「申請自託管(Hosting)」** 將引導您了解 NVIDIA 微服務(MICROSERVICES)的資訊，並為您提供一些註冊途徑（即早期存取/NVIDIA AI Enterprise 路徑）或進入通知清單（一般存取路徑）。

-   點擊 **「獲取 API 金鑰」** 將生成一個以「nvapi-」開頭的 API 金鑰，您可以透過網路請求將其提供給 API 端點(Endpoints)！

如果您要這樣做，您需要像這樣將 API 金鑰添加到 notebook 中：

In [1]:
# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

<br/>



**從您的課程環境：** 為了課程的目的，我們將透過在 `llm_client` 目錄中設置的伺服器（即 [**`llm_client/client_server.py`**](llm_client/client_server.py)）直接使用這些模型。圍繞其實作的細節超出了課程範圍，但將透過以下方式為您提供對模型選擇的無限存取：

-   公開一些端點(Endpoints)，將您的請求傳遞給選好的模型。

-   從 llm_client 微服務(MICROSERVICES)內填入 API 金鑰，這樣您就不會用完額度。

***相同的程式碼也可以用作實作您自己的 GenAI 閘道器服務（如 [`integrate.api.nvidia.com`](https://docs.api.nvidia.com/nim/reference/nvidia-embedding-2b-infer) 或 [`api.openai.com`](https://platform.openai.com/docs/api-reference/introduction)）的有用起點。***

### **4.1.** 手動 Python 請求


如我們之前所說，您可以使用 Python 的 `requests` 函式庫(library)與微服務(MICROSERVICES)或遠端 API 互動，通常會遵循以下過程：

-   **匯入函式庫(library)：** 我們首先匯入用於 HTTP 請求的 requests 和用於處理 JSON 資料的 json。
-   **API URL 和標頭：** 定義 API 端點(Endpoints)的 URL 和標頭，包括授權（API 金鑰）和資料格式偏好。
-   **資料負載：** 指定您要發送的資料；在這裡，這是一個簡單的查詢。
-   **發出請求：** 使用 `requests.post` 發送 POST 請求。您可以根據 API 的要求將 post 替換為 `get`、`put` 等。
-   **回應處理：** 檢查狀態碼以確定請求是否成功（200 表示成功），然後處理資料。

為了建立對服務的一些了解，我們可以嘗試看看它提供什麼樣的端點(Endpoints)和模型：

In [2]:
import requests

invoke_url = "http://llm_client:9000"
headers = {"content-type": "application/json"}

requests.get(invoke_url, headers=headers, stream=False).json()

{'/health': 'Health check endpoint to verify service is running.',
 '/hello': 'Returns a simple greeting, useful for initial testing.',
 '/': 'Lists all available endpoints with their descriptions.',
 '/set_keys': 'Update API keys for application',
 '/revert': 'Reverts API key to initial state',
 '/v1/models/{model:path}': 'Returns the listing of available models (or a single model)',
 '/v1/models': 'Returns the listing of available models (or a single model)',
 '/v1/{path:path}': 'Forwards requests based on the path to the appropriate OpenAPI endpoint.'}

In [3]:
import requests

invoke_url = "http://llm_client:9000/v1/models"
# invoke_url = "https://api.openai.com/v1/models"
# invoke_url = "https://integrate.api.nvidia.com/v1"
# invoke_url = "http://llm_client:9000/v1/models/mistralai/mixtral-8x7b-instruct-v0.1"
# invoke_url = "http://llm_client:9000/v1/models/mistralaimixtral-8x7b-instruct-v0.1"
headers = {
    "content-type": "application/json",
    # "Authorization": f"Bearer {os.environ.get('NVIDIA_API_KEY')}",
    # "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}",
}

print("Available Models:")
response = requests.get(invoke_url, headers=headers, stream=False)
# print(response.json())  ## <- Raw Response. Very Verbose
for model_entry in response.json().get("data", []):
    print(" -", model_entry.get("id"))

print("\nExample Entry:")
invoke_url = "http://llm_client:9000/v1/models/mistralai/mixtral-8x7b-instruct-v0.1"
requests.get(invoke_url, headers=headers, stream=False).json()

Available Models:
 - 01-ai/yi-large
 - abacusai/dracarys-llama-3.1-70b-instruct
 - ai21labs/jamba-1.5-large-instruct
 - ai21labs/jamba-1.5-mini-instruct
 - aisingapore/sea-lion-7b-instruct
 - baichuan-inc/baichuan2-13b-chat
 - databricks/dbrx-instruct
 - deepseek-ai/deepseek-coder-6.7b-instruct
 - deepseek-ai/deepseek-r1
 - deepseek-ai/deepseek-r1-0528
 - deepseek-ai/deepseek-r1-distill-llama-8b
 - deepseek-ai/deepseek-r1-distill-qwen-14b
 - deepseek-ai/deepseek-r1-distill-qwen-32b
 - deepseek-ai/deepseek-r1-distill-qwen-7b
 - google/codegemma-1.1-7b
 - google/codegemma-7b
 - google/gemma-2-27b-it
 - google/gemma-2-2b-it
 - google/gemma-2-9b-it
 - google/gemma-3-12b-it
 - google/gemma-3-1b-it
 - google/gemma-3-27b-it
 - google/gemma-3-4b-it
 - google/gemma-7b
 - google/recurrentgemma-2b
 - google/shieldgemma-9b
 - gotocompany/gemma-2-9b-cpt-sahabatai-instruct
 - ibm/granite-3.0-3b-a800m-instruct
 - ibm/granite-3.0-8b-instruct
 - ibm/granite-3.3-8b-instruct
 - ibm/granite-34b-code-instr

{'id': 'mistralai/mixtral-8x7b-instruct-v0.1',
 'object': 'model',
 'created': 735790403,
 'owned_by': 'mistralai'}

<br/>

我們在這門課程中不會在這個抽象(abstractions)層級上練習太多，但值得實作基本過程來確認，是的，這些請求正在透過我們的微服務(MICROSERVICES)以與伺服器遠端託管(Hosting)時幾乎相同的方式進入。您可以假設在課程的其餘部分，類似以下的互動正在您的客戶端幕後發生。

In [4]:
from getpass import getpass
import os

## Where are you sending your requests?
invoke_url = "http://llm_client:9000/v1/chat/completions"

## If you wanted to use your own API Key, it's very similar
# if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
#     os.environ["NVIDIA_API_KEY"] = getpass("NVIDIA_API_KEY: ")
# invoke_url = "https://integrate.api.nvidia.com/v1/chat/completions"

## If you wanted to use OpenAI, it's very similar
# if not os.environ.get("OPENAI_API_KEY", "").startswith("sk-"):
#     os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY: ")
# invoke_url = "https://api.openai.com/v1/models"

## Meta communication-level info about who you are, what you want, etc.
headers = {
    "accept": "text/event-stream",
    "content-type": "application/json",
    # "Authorization": f"Bearer {os.environ.get('NVIDIA_API_KEY')}",
    # "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}",
}

## Arguments to your server function
payload = {
    "model": "mistralai/mixtral-8x7b-instruct-v0.1",
    "messages": [{"role":"user","content":"Tell me hello in French"}],
    "temperature": 0.5,   
    "top_p": 1,
    "max_tokens": 1024,
    "stream": True                
}

In [5]:
import requests
import json

## Use requests.post to send the header (streaming meta-info) the payload to the endpoint
## Make sure streaming is enabled, and expect the response to have an iter_lines response.
response = requests.post(invoke_url, headers=headers, json=payload, stream=True)

## If your response is an error message, this will raise an exception in Python
try: 
    response.raise_for_status()  ## If not 200 or similar, this will raise an exception
except Exception as e:
    # print(response.json())
    print(response.json())
    raise e

## Custom utility to make live a bit easier
def get_stream_token(entry: bytes):
    """Utility: Coerces out ['choices'][0]['delta'][content] from the bytestream"""
    if not entry: return ""
    entry = entry.decode('utf-8')
    if entry.startswith('data: '):
        try: entry = json.loads(entry[5:])
        except ValueError: return ""
    return entry.get('choices', [{}])[0].get('delta', {}).get('content') or ""

## If the post request is honored, you should be able to iterate over 
for line in response.iter_lines():
    
    ## Without Processing: data: {"id":"...", ... "choices":[{"index":0,"delta":{"role":"assistant","content":""}...}...
    # if line: print(line.decode("utf-8"))

    ## With Processing: An actual stream of tokens printed one-after-the-other as they come in
    print(get_stream_token(line), end="")

In French, "hello" can be translated as "bonjour" (pronounced: bohn-zhoor). This is a formal way to greet someone during the daytime. If you're saying hello in the evening, you can use "bonsoir" (pronounced: bohn-swahr). For a casual or informal setting, you can use "salut" (pronounced: sah-luu), which can be used for both greetings and farewells.

Keep in mind that the pronunciation may vary slightly depending on the region and the speaker's accent. It's always a good idea to listen to native speakers and practice speaking the language to improve your pronunciation.

<br>

#### **[注意事項]**


**您可能會注意到聊天模型期望「messages」作為輸入(Intake)：**

如果您更習慣於原始 LLM 介面（如本地 HuggingFace 模型的介面），這可能是意外的，但對於 OpenAI 模型的使用者來說會看起來相當標準。透過強制執行受限制的介面而不是原始文字(Text)完成介面，服務可以對使用者能做什麼有更多控制。這個介面有很多優缺點，以下是一些值得注意的：

-   服務可能限制特定角色類型或參數的使用（即系統訊息限制、啟動訊息以獲得任意生成等）。
-   服務可能強制執行自訂提示(Prompt)格式並在幕後實作依賴聊天介面的額外選項。
-   服務可能使用更強的假設在推論(Inference)管線(Pipeline)中實作更深層的最佳化。
-   服務可能模仿另一個常見(popular)介面以利用現有的生態系統相容性。

所有這些都是有效的理由，在選擇或部署架構(Deployment)您自己的服務時，考慮哪些介面選項最適合您的特定使用案例是重要的。

**您可能會注意到查詢模型有兩種基本方式：**

您可以 **invoke 而不串流**，在這種情況下，服務回應將在完全計算後一次性到來。當您需要在做其他事情之前獲得模型的完整輸出時，這很好；例如，當您想要列印整個結果或將其用於下游任務時。回應主體將看起來像這樣：


```json
{
    "id": "d34d436a-c28b-4451-aa9c-02eed2141ed3",
    "choices": [{
        "index": 0,
        "message": { "role": "assistant", "content": "Bonjour! ..." },
        "finish_reason": "stop"
    }],
    "usage": {
        "completion_tokens": 450,
        "prompt_tokens": 152,
        "total_tokens": 602
    }
}
```

您也可以 **invoke 並串流**，在這種情況下，服務將發出一系列請求，直到發送最終請求。當您可以在服務回應可用時使用它們時，這很好（這對於直接向使用者列印輸出的語言模型組件非常好，因為它在生成時列印）。在這種情況下，回應主體將看起來更像這樣：

```json
data:{"id":"...","choices":[{"index":0,"delta":{"role":"assistant","content":"Bon"},"finish_reason":null}]}
data:{"id":"...","choices":[{"index":0,"delta":{"role":"assistant","content":"j"},"finish_reason":null}]}
...
data:{"id":"...","choices":[{"index":0,"delta":{"role":"assistant","content":""},"finish_reason":"stop"}]}
data:[DONE]
``



這兩個選項都可以使用 Python 的 `requests` 函式庫(library)相對容易地完成，但按原樣使用介面將導致大量重複程式碼。幸運的是，我們有一些系統使這顯著更容易使用並整合到更大的專案中！

<br/>

### **4.2.** OpenAI 客戶端請求


知道這個介面存在是好的，但按原樣使用它將導致大量重複程式碼和額外複雜性。幸運的是，我們有一些系統使這顯著更容易使用並整合到更大的專案中！使用請求之上的一個抽象(abstractions)層是使用更有主導性的客戶端，如 OpenAI 的客戶端。由於 NVIDIA 和 OpenAI 都訂閱 OpenAPI 規範，我們可以借用他們的客戶端。請注意，在幕後，相同的過程仍在進行，可能由像 [**httpx**](https://github.com/projectdiscovery/httpx) 或 [**aiohttp**](https://github.com/aio-libs/aiohttp) 這樣的較低階客戶端進行。

In [6]:
## Using General OpenAI Client
from openai import OpenAI

# client = OpenAI()  ## Assumes OPENAI_API_KEY is set

# client = OpenAI(
#     base_url = "https://integrate.api.nvidia.com/v1",
#     api_key = os.environ.get("NVIDIA_API_KEY", "")
# )

client = OpenAI(
    base_url = "http://llm_client:9000/v1",
    api_key = "I don't have one"
)

completion = client.chat.completions.create(
    model="mistralai/mixtral-8x7b-instruct-v0.1",
    # model="gpt-4-turbo-2024-04-09",
    messages=[{"role":"user","content":"Hello World"}],
    temperature=1,
    top_p=1,
    max_tokens=1024,
    stream=True,
)

## Streaming with Generator: Results come out as they're generated
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

Hello! It's nice to meet you. "Hello World" is often the first program that people write when they are learning a new programming language. It's a simple program that outputs the text "Hello World" to the console. I'm here to help answer any questions you have about programming or technology, so feel free to ask me anything!

In [7]:
## Non-Streaming: Results come from server when they're all ready
completion = client.chat.completions.create(
    model="mistralai/mixtral-8x7b-instruct-v0.1",
    # model="gpt-4-turbo-2024-04-09",
    messages=[{"role":"user","content":"Hello World"}],
    temperature=1,
    top_p=1,
    max_tokens=1024,
    stream=False,
)

completion

ChatCompletion(id='cmpl-58d997eb6b244875b6088b4832b54748', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! It\'s nice to meet you. "Hello World" is often the first program that people write when they are learning a new programming language. It\'s a simple program that outputs the text "Hello World" to the console. I\'m here to help answer any questions you have about programming or technology, so feel free to ask me anything!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), stop_reason=None)], created=1750230418, model='mistralai/mixtral-8x7b-instruct-v0.1', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=73, prompt_tokens=11, total_tokens=84, completion_tokens_details=None, prompt_tokens_details=None))

<br/>

### **4.3.** ChatNVIDIA 客戶端請求


到目前為止，我們已經看到通訊在兩個想法(abstractions)層級上發生：**原始請求**和 **API 客戶端**。在這門課程中，我們將希望使用稱為 LangChain 的框架進行 LLM 流程協調管理(Orchestration)，所以我們需要上升一個抽象(abstractions)層級到 **框架連接器(Connector)**。

**連接器(Connector)**的目標是將任意 API 從其原生核心轉換為目標程式碼庫期望的 API。在這門課程中，我們將希望利用 LangChain 蓬勃發展的以鏈(Chain)為中心的生態系統，但原始的 `requests` API 不會帶我們到那裡。在幕後，每個不在本地託管(Hosting)的 LangChain 聊天模型都必須依賴這樣的 API，但面向開發者的 API 是一個更乾淨的 [`LLM` 或 `SimpleChatModel` 風格介面](https://python.langchain.com/v0.1/docs/modules/model_io/)，具有預設參數和一些簡單的實用函式(function)，如 `invoke` 和 `stream`。

為了開始我們對 LangChain 介面的探索，我們將使用 `ChatNVIDIA` 連接器(Connector)與我們的 `chat/completions` 端點(Endpoints)介面。這個模型是 LangChain 擴展生態系統的一部分，可以透過 `pip install langchain-nvidia-ai-endpoints` 在本地安裝。



In [8]:
## Using ChatNVIDIA
from langchain_nvidia_ai_endpoints import ChatNVIDIA

## NVIDIA_API_KEY pulled from environment
llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", mode="open", base_url="http://llm_client:9000/v1")
llm.invoke("Hello World")

AIMessage(content='Hello! It\'s nice to meet you. "Hello World" is often the first program that people write when they are learning a new programming language. It\'s a simple program that outputs the text "Hello World" to the console. I\'m here to help answer any questions you have about programming or technology, so feel free to ask me anything!', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Hello! It\'s nice to meet you. "Hello World" is often the first program that people write when they are learning a new programming language. It\'s a simple program that outputs the text "Hello World" to the console. I\'m here to help answer any questions you have about programming or technology, so feel free to ask me anything!', 'token_usage': {'prompt_tokens': 11, 'total_tokens': 84, 'completion_tokens': 73}, 'finish_reason': 'stop', 'model_name': 'mistralai/mixtral-8x7b-instruct-v0.1'}, id='run--8b4eb261-4142-4ffa-94de-32491e34264a-0', usage_metadata={'input_tokens'

In [9]:
llm._client.last_inputs

{'url': 'http://llm_client:9000/v1/chat/completions',
 'headers': {'Accept': 'application/json',
  'Authorization': 'Bearer **********',
  'User-Agent': 'langchain-nvidia-ai-endpoints'},
 'json': {'messages': [{'role': 'user', 'content': 'Hello World'}],
  'model': 'mistralai/mixtral-8x7b-instruct-v0.1',
  'max_tokens': 1024,
  'stream': False}}

In [10]:
# llm._client.last_response
llm._client.last_response.json()

{'id': 'cmpl-3716d625089147c3a9e1fd7411745574',
 'object': 'chat.completion',
 'created': 1750230420,
 'model': 'mistralai/mixtral-8x7b-instruct-v0.1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! It\'s nice to meet you. "Hello World" is often the first program that people write when they are learning a new programming language. It\'s a simple program that outputs the text "Hello World" to the console. I\'m here to help answer any questions you have about programming or technology, so feel free to ask me anything!'},
   'logprobs': None,
   'finish_reason': 'stop',
   'stop_reason': None}],
 'usage': {'prompt_tokens': 11, 'total_tokens': 84, 'completion_tokens': 73}}

<br/>

#### **[注意事項]**


-   **課程使用 `ai-endpoints` 連接器(Connector)的修改分支，具有幾個對我們課程環境更有用的功能。** 這些功能尚未在主版本中，正在與來自 [**LlamaIndex**](https://docs.llamaindex.ai/en/stable/examples/embeddings/nvidia/) 和 [**Haystack**](https://docs.haystack.deepset.ai/docs/nvidiagenerator) 對應方的其他開發和要求一起主動整合。

-   **ChatNVIDIA 預設為 `llm_client` 微服務(MICROSERVICES)，因為我們設置了一些環境變數來實現這一點**：



In [11]:
import os

{k:v for k,v in os.environ.items() if k.startswith("NVIDIA_")}
## Translation: Use the base_url of llm_client:9000 for the requests,
## and use "open"api-spec access for model discovery and url formats

{'NVIDIA_DEFAULT_MODE': 'open', 'NVIDIA_BASE_URL': 'http://llm_client:9000/v1'}

<br/>


**在整個課程中，請隨意試用您選擇的模型。** 以下是作為這門課程一部分提供的模型選擇，其中一個選擇應該在任何給定時間運作。

您也可以自由嘗試其他模型，儘管您可能需要升級 `langchain_nvidia_ai_endpoints` 函式庫(library)並提供您自己的金鑰。

In [12]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

model_list = ChatNVIDIA.get_available_models()

for model_card in model_list:
    model_name = model_card.id
    ## If you want to, might be a good idea to not go through EVERY model
    if not any([keyword in model_name for keyword in ["meta/llama"]]): continue
    if "405b" in model_name: continue
    if "embed" in model_name: continue
    
    llm = ChatNVIDIA(model=model_name)
    print(f"TRIAL: {model_name}")
    try: 
        for token in llm.stream("Tell me about yourself! 2 sentences.", max_tokens=100):
            print(token.content, end="")
    except Exception as e: 
        print(f"EXCEPTION: {e}")    ## If some models fail, feel free to use others
    except KeyboardInterrupt:
        print(f"Stopped manually")  ## Feel free to hit square while running
        break
    print("\n\n" + "="*84)

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:237: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(


TRIAL: meta/llama-3.1-70b-instruct
I'm an artificial intelligence language model, which means I'm a computer program designed to understand and respond to human language, generating text based on the input I receive. I'm a large language model, I don't have a personal identity or consciousness, but I'm here to help answer your questions, provide information, and assist with tasks to the best of my abilities!

TRIAL: meta/llama-3.1-8b-instruct
I'm an artificial intelligence designed to assist and communicate with humans, and I don't have a physical body or personal identity, but I can provide information and answer questions on a wide range of topics. I'm a large language model, trained on a massive dataset of text to generate human-like responses, and I'm here to help with any questions or topics you'd like to discuss!

TRIAL: meta/llama-3.2-11b-vision-instruct
I'm an AI language model, designed to provide information and answer questions on a wide range of topics. I don't have a perso



<br>

## **第五部分：** 總結


這個 notebook 的目標是提供一些圍繞 LLM 服務託管(Hosting)策略的討論，並向您介紹 AI 基礎模型端點(Endpoints)。在這個過程中，我們希望您對如何從邊緣裝置(Edge Device)提供和存取遠端 LLM 系統有直觀的理解！

<font color="#76b900">**做得很好！**</font>


**下一步：**


1.  **[可選]** 重新訪問 notebook 頂部的**「值得思考的問題」部分**，並思考一些可能的答案。
<br>



<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

